# Spotify data analysis: A retrospective

Every December since 2016, Spotify users discover their "Spotify Wrapped". The latter provides a compilation of data about their activity on the platform over the past year: top artists, top songs, top genres, etc. They get a deep dive into their most memorable listening moments of the year.

Objectives:
* JSON : Import several JSON files in an elegant way in Python
* SQL : 
  * Write SQL queries in Python
  * Demonstrate my abilities to code in SQL -> WINDOW FUNCTIONS, JOINS, etc
* PYTHON : Make a summary visual report in Python (hvplot)

In [1]:
# Import all relevant Python packages
import glob
import os
import pandas as pd
import json
import string
import sqlite3

## 1. Import and prepare the Spotify data (JSON files)

*Objective* : I received my Spotify data in different batches (8 JSON files). My goal is to import all of them at once in a Pandas dataframe, and remove every private information (IP address, device information, etc).  
*Inspiration for the code below* : https://stackoverflow.com/questions/41857659/python-pandas-add-filename-column-csv

### 1.1. Importing the data

In [2]:
# Step 1 : I look for all JSON files available in my Windows folder (where I stored my Spotify data)

path_to_json = 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData/' #local path to my JSON files folder 
json_pattern = os.path.join(path_to_json,'*.json') #looking for every JSON file
globbed_files = glob.glob(json_pattern) #listing all the JSON files paths
globbed_files #overview

['C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_0.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_1.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_2.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_3.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_4.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_5.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endsong_6.json',
 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData\\endvideo.json']

Comment: My Windows folder contains 8 different JSON files. 
* 7 start with the name "endsong"
* 1 is focused on videos (endvideo)

In [3]:
# Step 2: I open and read every JSON file thanks to a for loop. 
   # I also add a column "filename" that contains the name of each file
   # Finally, I store all data in a "data" list

data = [] # empty list (initialization) 
for json in globbed_files:
    frame = pd.read_json(json) 
    frame['filename'] = os.path.basename(json) 
    data.append(frame) 

In [4]:
# Step 3: Export the data to csv format
bigframe = pd.concat(data, ignore_index=True) #dont want pandas to try an align row indexes
bigframe.to_csv("spotify_data.csv", sep = ';')

C:\Users\margo\AppData\Local\Temp\ipykernel_22292\1933132580.py:2: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  bigframe = pd.concat(data, ignore_index=True) #dont want pandas to try an align row indexes


### 1.2. Preparing the data

In [5]:
# Step 4 : Remove private information (username, user agent, device, IP address) 
spotify_data = pd.read_csv("spotify_data.csv", sep = ';', index_col = 0)

# Blur device information
spotify_data["device"] = spotify_data["platform"].str[:10]

# Delete columns that share too much information
to_delete = ['username', 'ip_addr_decrypted', 'user_agent_decrypted', 'platform']
spotify_data.drop(to_delete, axis=1, inplace=True)

In [6]:
# Step 5 : Add time variables that will be useful for data analysis
spotify_data['year'] = pd.to_datetime(spotify_data['ts']).dt.strftime('%Y')
spotify_data['year_month'] = pd.to_datetime(spotify_data['ts']).dt.strftime('%Y-%m')
spotify_data["date"] = pd.to_datetime(spotify_data["ts"]).dt.strftime('%Y-%m-%d')

In [8]:
# Step 6: Rename some variables whose names are unclear and/or too long
spotify_data = spotify_data.rename(columns = {'master_metadata_track_name' : 'track_name',
                                   'master_metadata_album_artist_name' : 'artist_name',
                                   'master_metadata_album_album_name' : 'album_name'
                                             })

#End results
pd.set_option('display.max_columns', None)
spotify_data.sort_values('ts', ascending = False).head(5)

,ts,ms_played,conn_country,track_name,artist_name,album_name,spotify_track_uri,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode,filename,device,year,year_month,date
61375,2022-11-24T22:42:26Z,788584,FR,NaN,NaN,NaN,NaN,Journal 07h00 du mardi 22 novembre 2022,Journal de 07h00,spotify:episode:4Adwt7pETOXPNplhooHLa1,clickrow,endplay,False,NaN,0.0,1.669328e+12,False,endsong_3.json,Android OS,2022,2022-11,2022-11-24
88700,2022-11-24T22:18:34Z,155703,FR,Dancing With The Devil,ISBANKY,Dancing With The Devil,spotify:track:0ChUbF2A6VjPVRFOKJ8VSN,NaN,NaN,NaN,trackdone,logout,False,NaN,0.0,1.669325e+12,False,endsong_5.json,Android OS,2022,2022-11,2022-11-24
36788,2022-11-24T21:29:29Z,123750,FR,Pharaoh's Wrath,Feerix,Pharaoh's Wrath,spotify:track:1XQQwnbfGqezphAHjaVxXR,NaN,NaN,NaN,trackdone,trackdone,False,NaN,0.0,1.669325e+12,False,endsong_2.json,Android OS,2022,2022-11,2022-11-24
7564,2022-11-24T21:27:24Z,187306,FR,Ma solitude,Kendji Girac,Ensemble,spotify:track:2lZW1J1ScVcV8qHMkHN64u,NaN,NaN,NaN,clickrow,trackdone,False,NaN,0.0,1.669325e+12,False,endsong_0.json,Android OS,2022,2022-11,2022-11-24
89308,2022-11-24T21:24:18Z,17268,FR,Poison Love,Dreamcatcher,[Dystopia : Road to Utopia],spotify:track:7C3cUybMDDF7PKyFcDWouF,NaN,NaN,NaN,trackdone,endplay,False,NaN,0.0,1.669325e+12,False,endsong_5.json,Android OS,2022,2022-11,2022-11-24


## 2. Set up the SQL connection

*Objective* : The spotify_data dataframe will be analyzed thanks to SQL queries. The goal is indeed to mimick data analysis in a Datawharehouse like Teradata or Google Big Query. To do so, I must create a SQL connection beforehand.

In [9]:
# Create the SQLITE3 connection
cnx = sqlite3.connect(':memory:')

# Transform the Pandas dataframe "spotify_data" to a SQL table "spotify"
spotify_data.to_sql(name='spotify', con=cnx)

97998

## 3. Understand the dataset

*Objective*: I perform some simple exploratory data analyses to better understand the dataset I retrieved in the previous steps. The goal is to answer some simple questions:
* What is the total volumetry? How is it distributed per JSON file and year ?
* What is the period considered (min and max dates) ?
* Are there problems (NULLs, duplicates, etc) ?

In [10]:
# Total volumetry: About 100k rows in total
volumetry = pd.read_sql('''SELECT COUNT(*) AS nb_rows 
                        FROM spotify''', cnx)
print(volumetry)

   nb_rows
0    97998


In [19]:
# Volumetry per file: 
    # Every JSON file contains about 15,800 rows (except the video and last endsong files)
    # The data span from February 2020 to end November 2022
    # Each file contains about the same proportion of music tracks (98%) / podcasts episodes (2%) 
    # - except the endvideo file that contains only video podcasts
volumetry_file = pd.read_sql('''SELECT filename, 
                                 MIN(date) AS min_date,
                                 MAX(date) AS max_date,
                                 COUNT(*) AS nb_rows,
                                 SUM(CASE WHEN track_name IS NOT NULL THEN 1 ELSE 0 END) AS music_tracks,
                                 SUM(CASE WHEN episode_name IS NOT NULL THEN 1 ELSE 0 END) AS podcast_eps,
                                 ROUND(CAST(SUM(CASE WHEN track_name IS NOT NULL THEN 1 ELSE 0 END) AS FLOAT)
                                   / CAST(COUNT(*) AS FLOAT),2) AS pct_music_tracks,
                                 ROUND(CAST(SUM(CASE WHEN episode_name IS NOT NULL THEN 1 ELSE 0 END) AS FLOAT) /
                                 CAST(COUNT(*) AS FLOAT), 2) AS pct_podcast
                             FROM spotify
                             GROUP BY 1''', cnx)

print(volumetry_file)

         filename    min_date    max_date  nb_rows  music_tracks  podcast_eps  \
0  endsong_0.json  2020-02-02  2022-11-24    15851         15485          357   
1  endsong_1.json  2020-02-02  2022-11-24    15855         15510          332   
2  endsong_2.json  2020-02-02  2022-11-24    15849         15509          322   
3  endsong_3.json  2020-02-02  2022-11-24    15838         15507          317   
4  endsong_4.json  2020-02-02  2022-11-24    15834         15504          320   
5  endsong_5.json  2020-02-02  2022-11-24    15833         15472          351   
6  endsong_6.json  2020-02-03  2022-11-24     2900          2849           51   
7   endvideo.json  2021-07-30  2022-11-22       38             0           38   

   pct_music_tracks  pct_podcast  
0              0.98         0.02  
1              0.98         0.02  
2              0.98         0.02  
3              0.98         0.02  
4              0.98         0.02  
5              0.98         0.02  
6              0.98      

## 4. Data analysis

### 4.1 Yearly listening habits

In [32]:
## 4.1. Listening habits
listening_year = \
    pd.read_sql('''
             SELECT year, nb_sessions, artists, music_tracks, podcasts, podcast_shows,
                 ROUND(CAST(nb_sessions AS FLOAT) / 
                       CAST(LAG(nb_sessions) OVER (ORDER BY year) AS FLOAT),2) -1 AS evol_nb_sessions,
                 ROUND(CAST(artists AS FLOAT) / 
                       CAST(LAG(artists) OVER (ORDER BY year) AS FLOAT),2) -1 AS evol_artists, 
                 ROUND(CAST(music_tracks AS FLOAT) / 
                       CAST(LAG(music_tracks) OVER (ORDER BY year) AS FLOAT),2) -1 AS evol_music_tracks,
                 ROUND(CAST(podcasts AS FLOAT) / 
                       CAST(LAG(podcasts) OVER (ORDER BY year) AS FLOAT), 2) -1 AS evol_podcasts, 
                 ROUND(CAST(podcast_shows AS FLOAT) / 
                       CAST(LAG(podcast_shows) OVER (ORDER BY year) AS FLOAT), 2) -1 AS evol_podcast_shows
             FROM (
             SELECT year, 
                   COUNT(*) AS nb_sessions,
                   COUNT(DISTINCT artist_name) AS artists, 
                   COUNT(DISTINCT spotify_track_uri) AS music_tracks,
                   COUNT(DISTINCT spotify_episode_uri) AS podcasts,
                   COUNT(DISTINCT episode_show_name) AS podcast_shows
               FROM spotify
               GROUP BY year
               ORDER BY year DESC
             ) subquery
             ORDER BY year DESC''', cnx)
print(listening_year)

   year  nb_sessions  artists  music_tracks  podcasts  podcast_shows  \
0  2022        44371     1465          4896       916             44   
1  2021        33696      782          2643       622             37   
2  2020        19931      693          2628        45             13   

   evol_nb_sessions  evol_artists  evol_music_tracks  evol_podcasts  \
0              0.32          0.87               0.85           0.47   
1              0.69          0.13               0.01          12.82   
2               NaN           NaN                NaN            NaN   

   evol_podcast_shows  
0                0.19  
1                1.85  
2                 NaN  


In [ ]:
## 4.1. Favorite tracks and artists

Opérations à faire : 
* Transformer certaines variables : millisecondes en secondes et minutes

In [ ]:
Questions : 
    * Combien d'artistes / chansons écoutés par an ? Et quelle évolution (cf. playlists : j'en écoutais pas avant -> normalement, j'écoute de plus en plus d'artistes différents)
    * Quels genres de musique ? -> essayer de récupérer l'info d'une source externe
    * Combien de morceaux écoutés par jour ?
    * Temps moyen d'écoute par jour ?
    * Durée de mes morceaux populaires : de plus en plus courts ?
    * Est-ce que je suis maniaque d'un artiste -> j'écoute tout de lui ou alors j'aime seulement un seul morceau de l'artiste à mort ?
    * Est-ce que je "zappe" beaucoup ? 
    * Suis-je une utilisatrice active : jours consécutifs d'écoute dans le mois / la semaine ?
    * Quand j'écoute la musique : matin, midi, soir ? -> transports en commun, travail, week-end ?
    * Quels sont mes tops 10 artistes ? Comment ont-ils évolué ? -> toujours les mêmes ou pas ?
    * Même question pour mes meilleures chansons : ça bouge ou pas ? Quelles sont leurs caractéristiques : même artiste, même genre, même durée,... ?
    * Quand sont sortis les morceaux populaires ? -> Est-ce que j'écoute de la musique new ou bien je commence à être nostalgique de vieux morceaux ?
    * Qu'en est-il des podcasts ? -> Je les écoute avant de m'endormir ?
    * Combien de podcasts par jour ?
    * Est-ce que j'utilise beaucoup certaines features de Spotify : mode incognito, mode hors ligne, ... ?

In [ ]:
# Step 2 : SQL connection
import sqlite3

# Set up SQL connexion
sql_connect = sqlite3.connect('spotify_data.db')

# To execute some SQL command
cursor = sql_connect.cursor()

# Create database
#cursor.execute('''CREATE TABLE spotify_data (
 #   ts timestamp,
  #  username text)''') 

#cursor.execute("INSERT INTO spotify_data VALUES ('2022-07-17T06:07:42Z', 'v7x27nfjb2dri60b7jzl159rl')")

cursor.execute("SELECT * FROM spotify_data;")
print(cursor.fetchone())

sql_connect.commit()

sql_connect.close()

In [ ]:
# Test 2 : Je récupère la liste des fichiers JSON dans mon dossier
import os, json
import pandas as pd

path_to_json = 'C:/Users/margo/Documents/Documents/Formation/Github/Spotify/MyData/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files) 

# Puis j'essaie d'importer chaque file une par une 
for file in json_files :
    df = pd.read_json(path_to_json + file)
    
df

In [ ]:
spotify_data = pd.DataFrame(columns=['ts', 'username', 'platform',
                                    'ms_played', 'conn_country', 'user_agent_decrypted',
                                     'master_metadata_track_name',
                                     'master_metadata_album_artist_name',
                                     'master_metadata_album_album_name',
                                     'spotify_track_uri',
                                     'episode_name',
                                     'episode_show_name',
                                     'spotify_episode_uri',
                                     'reason_start',
                                     'reason_end',
                                     'shuffle', 'skipped', 'offline', 'offline_timestamp'
                                    ])
spotify_data.head(5)
#print spotify_json['features'][0]['geometry']